<a href="https://colab.research.google.com/github/RuqsarFirdous/BIG-DATA-ANALYSIS/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
# Create Spark session
spark=SparkSession.builder.appName("FlightDelayAnalysis").getOrCreate()

## 📂 Data Loaded
We are using 3 datasets:
- **Flights**: flight details with airline code, delays, airports.
- **Airlines**: airline code and full name.
- **Airports**: airport codes, city, state.


In [7]:
# Load flights data
flights_df=spark.read.csv("flights.csv",header=True,inferSchema=True)

In [8]:
# Load airlines data
airlines_df=spark.read.csv("airlines.csv",header=True,inferSchema=True)

In [10]:
# Load airports data
airports_df=spark.read.csv("airports.csv",header=True,inferSchema=True)

In [14]:
print("Flights Data:")
flights_df.show(5)

Flights Data:
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+---------

In [15]:
print("Airlines data:")
airlines_df.show(5)

Airlines data:
+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
+---------+--------------------+
only showing top 5 rows



In [16]:
print("Airports data:")
airports_df.show(5)

Airports data:
+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



In [17]:
print("Flights Schema:")
flights_df.printSchema()

Flights Schema:
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELA

## ✈️ Join Flights + Airlines
We join flights with airline names to get readable airline names for analysis.


In [23]:
from pyspark.sql.functions import col

flights_with_airline = flights_df.join(
    airlines_df,
    flights_df.AIRLINE == airlines_df.IATA_CODE,
    how="left"
)

flights_with_airline=flights_with_airline.select(
    flights_df.AIRLINE.alias("AIRLINE_CODE"),
    airlines_df.AIRLINE.alias("AIRLINE_NAME"),
    flights_df.ORIGIN_AIRPORT,
    flights_df.DESTINATION_AIRPORT,
    flights_df.DEPARTURE_DELAY,
    flights_df.ARRIVAL_DELAY
)

flights_with_airline.show(5)


+------------+--------------------+--------------+-------------------+---------------+-------------+
|AIRLINE_CODE|        AIRLINE_NAME|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DEPARTURE_DELAY|ARRIVAL_DELAY|
+------------+--------------------+--------------+-------------------+---------------+-------------+
|          AS|Alaska Airlines Inc.|           ANC|                SEA|            -11|          -22|
|          AA|American Airlines...|           LAX|                PBI|             -8|           -9|
|          US|     US Airways Inc.|           SFO|                CLT|             -2|            5|
|          AA|American Airlines...|           LAX|                MIA|             -5|           -9|
|          AS|Alaska Airlines Inc.|           SEA|                ANC|             -1|          -21|
+------------+--------------------+--------------+-------------------+---------------+-------------+
only showing top 5 rows



## 🛫 Join + Airports
Next, we join airports to add the **city** and **state** for each flight's origin airport.


In [26]:
# join flights_with_airline + airports_df

flights_with_origin_airport=flights_with_airline.join(
    airports_df,
    flights_with_airline.ORIGIN_AIRPORT==airports_df.IATA_CODE,
    how="left"
)

flights_with_origin_airport = flights_with_origin_airport.select(
    "AIRLINE_NAME",
    "ORIGIN_AIRPORT",
    "CITY",
    "STATE",
    "DEPARTURE_DELAY"
)
flights_with_origin_airport.show(5)

+--------------------+--------------+-------------+-----+---------------+
|        AIRLINE_NAME|ORIGIN_AIRPORT|         CITY|STATE|DEPARTURE_DELAY|
+--------------------+--------------+-------------+-----+---------------+
|Alaska Airlines Inc.|           ANC|    Anchorage|   AK|            -11|
|American Airlines...|           LAX|  Los Angeles|   CA|             -8|
|     US Airways Inc.|           SFO|San Francisco|   CA|             -2|
|American Airlines...|           LAX|  Los Angeles|   CA|             -5|
|Alaska Airlines Inc.|           SEA|      Seattle|   WA|             -1|
+--------------------+--------------+-------------+-----+---------------+
only showing top 5 rows



## 🧹 Fill Missing Data
Missing values are replaced with:
- `"Unknown Airline"`
- `"Unknown City"`
- `0` for delay columns


In [28]:
# Fill missing airline names with 'Unknown'
filled_flights = flights_with_origin_airport.fillna({
    "AIRLINE_NAME": "Unknown Airline",
    "CITY": "Unknown City",
    "STATE": "Unknown State",
    "DEPARTURE_DELAY": 0
})

filled_flights.show(5)

+--------------------+--------------+-------------+-----+---------------+
|        AIRLINE_NAME|ORIGIN_AIRPORT|         CITY|STATE|DEPARTURE_DELAY|
+--------------------+--------------+-------------+-----+---------------+
|Alaska Airlines Inc.|           ANC|    Anchorage|   AK|            -11|
|American Airlines...|           LAX|  Los Angeles|   CA|             -8|
|     US Airways Inc.|           SFO|San Francisco|   CA|             -2|
|American Airlines...|           LAX|  Los Angeles|   CA|             -5|
|Alaska Airlines Inc.|           SEA|      Seattle|   WA|             -1|
+--------------------+--------------+-------------+-----+---------------+
only showing top 5 rows



### 1️⃣ Busiest Origin Cities

In [29]:
# Count total flights per city
filled_flights.groupBy("CITY").count().orderBy("count",ascending=False).show(10)

+-----------------+------+
|             CITY| count|
+-----------------+------+
|     Unknown City|486165|
|          Chicago|366770|
|          Atlanta|346836|
|Dallas-Fort Worth|239551|
|          Houston|198664|
|           Denver|196055|
|      Los Angeles|194673|
|         New York|193416|
|    San Francisco|148008|
|          Phoenix|146815|
+-----------------+------+
only showing top 10 rows



###2️⃣ Count total flights per airline

In [30]:
# Total flights per airline
filled_flights.groupBy("AIRLINE_NAME").count().orderBy("count", ascending=False).show(10)

+--------------------+-------+
|        AIRLINE_NAME|  count|
+--------------------+-------+
|Southwest Airline...|1261855|
|Delta Air Lines Inc.| 875881|
|American Airlines...| 725984|
|Skywest Airlines ...| 588353|
|Atlantic Southeas...| 571977|
|United Air Lines ...| 515723|
|American Eagle Ai...| 294632|
|     JetBlue Airways| 267048|
|     US Airways Inc.| 198715|
|Alaska Airlines Inc.| 172521|
+--------------------+-------+
only showing top 10 rows



### 3️⃣ Average departure delay per city

In [31]:
# Average departure delay for each city
filled_flights.groupBy("CITY").avg("DEPARTURE_DELAY").orderBy("avg(DEPARTURE_DELAY)", ascending=False).show(10)


+--------------------+--------------------+
|                CITY|avg(DEPARTURE_DELAY)|
+--------------------+--------------------+
|    Marthas Vineyard|   25.90731707317073|
|             Hyannis|  22.903614457831324|
|            St Cloud|  17.566265060240966|
|          North Bend|  17.130909090909093|
|            Gustavus|  16.948051948051948|
|               Agana|  16.547904191616766|
|               Aspen|  16.244525547445257|
|           Nantucket|    16.1890243902439|
|       Crescent City|  15.947368421052632|
|Beaumont/Port Arthur|  15.936279547790338|
+--------------------+--------------------+
only showing top 10 rows



### 4️⃣ Average departure delay per airline

In [32]:
# Average departure delay for each airline
filled_flights.groupBy("AIRLINE_NAME").avg("DEPARTURE_DELAY").orderBy("avg(DEPARTURE_DELAY)", ascending=False).show(10)


+--------------------+--------------------+
|        AIRLINE_NAME|avg(DEPARTURE_DELAY)|
+--------------------+--------------------+
|    Spirit Air Lines|   15.68327383944317|
|United Air Lines ...|   14.26220664969373|
|Frontier Airlines...|   13.27060856928971|
|     JetBlue Airways|  11.333044995656211|
|Southwest Airline...|  10.450107183471951|
|American Eagle Ai...|    9.63204268375465|
|      Virgin America|   8.947094648078446|
|American Airlines...|   8.773519802089302|
|Atlantic Southeas...|    8.49219111957299|
|Skywest Airlines ...|    7.67823058605973|
+--------------------+--------------------+
only showing top 10 rows



### 5️⃣ Average arrival delay per city

In [ ]:
# Average arrival delay for each city
filled_flights.groupBy("CITY").avg("ARRIVAL_DELAY").orderBy("avg(ARRIVAL_DELAY)", ascending=False).show(10)


### 7️⃣ Filter big delays (> 1 hour)

In [33]:
# Flights delayed more than 60 mins
big_delays = filled_flights.filter(filled_flights.DEPARTURE_DELAY > 60)
big_delays.show(5)

print("Total flights delayed more than 60 mins:", big_delays.count())


+--------------------+--------------+-----------------+-----+---------------+
|        AIRLINE_NAME|ORIGIN_AIRPORT|             CITY|STATE|DEPARTURE_DELAY|
+--------------------+--------------+-----------------+-----+---------------+
|    Spirit Air Lines|           PHX|          Phoenix|   AZ|             72|
|     JetBlue Airways|           SJU|         San Juan|   PR|             95|
|     JetBlue Airways|           SJU|         San Juan|   PR|             72|
|American Airlines...|           DFW|Dallas-Fort Worth|   TX|            108|
|Delta Air Lines Inc.|           PHX|          Phoenix|   AZ|            148|
+--------------------+--------------+-----------------+-----+---------------+
only showing top 5 rows

Total flights delayed more than 60 mins: 324171


### Total Cancelled vs Not Cancelled

In [34]:
## Use the original flights_df for this:
flights_df.groupBy("CANCELLED").count().show()

+---------+-------+
|CANCELLED|  count|
+---------+-------+
|        1|  89884|
|        0|5729195|
+---------+-------+



### Flights per Month

In [35]:
flights_df.groupBy("MONTH").count().orderBy("MONTH").show()

+-----+------+
|MONTH| count|
+-----+------+
|    1|469968|
|    2|429191|
|    3|504312|
|    4|485151|
|    5|496993|
|    6|503897|
|    7|520718|
|    8|510536|
|    9|464946|
|   10|486165|
|   11|467972|
|   12|479230|
+-----+------+



### 8️⃣ Save one result to CSV

In [36]:
# Save avg departure delay by airline
avg_delay_airline = filled_flights.groupBy("AIRLINE_NAME").avg("DEPARTURE_DELAY")
avg_delay_airline.write.csv("avg_departure_delay_by_airline.csv")

## ✅ Conclusion
- We analyzed US flights for delays.
- We joined **big data**: multiple CSVs.
- We filled missing data.
- We grouped and found the busiest cities, worst airlines, and biggest delays.
- We saved results for reporting.

🎉 **This is a complete Big Data PySpark mini project!**
